# Starbucks Capstone Challenge
Notebook 3 of 4

### Promotions and Transactions
Now we must transform our transcript.json file to make the data into a format that reflects whether the customer's purchases were influenced by a given promotion or not. This is a non-trivial exercise due to the original format of our data. 

In [1]:
import pandas as pd
import numpy as np
import math
from IPython.display import display, HTML

prefix = 'segmentation_data'

def import_csvs(prefix, filename):
    df = pd.read_csv(prefix + filename)
    df.index = df['id']
    df = df.drop(columns ='id')    
    return df
    
customers = import_csvs(prefix,'/customers.csv')

print("customers: {} rows and {} columns".format(customers.shape[0],customers.shape[1]))

customers: 14288 rows and 6 columns


In [2]:
import json

prefix = 'raw_data'

# read in the json files
portfolio = pd.read_json(prefix + '/portfolio.json', orient='records', lines=True)
transcript = pd.read_json(prefix + '/transcript.json', orient='records', lines=True)

# Assumptions
In order to proceed with our analysis, we will need to make several assumptions. 
- Since customers can unknowingly complete a promotion, we will not use completion of the promotion in our analysis. Instead, we will consider the act of viewing the promotion as the influencer to the customer's spending. In this manner, we will count of the views of different promotions during the period and compare this with overall spending for each customer. 
- We will not attempt to consider the time periods and whether purchases were made towards a specific offer. We will also not consider the time between receiving an offer, viewing it, and completing it.

In [3]:
print(transcript.event.unique())

['offer received' 'offer viewed' 'transaction' 'offer completed']


In [4]:
offer_views = transcript[transcript['event']=='offer viewed']
print('Transcript has {} rows.\nWe will only use the {} rows for offer receipts'.format(transcript.shape[0],offer_views.shape[0]))
display(HTML(offer_views.iloc[0:5].to_html()))

Transcript has 306534 rows.
We will only use the 57725 rows for offer receipts


,person,event,value,time
12650,389bc3fa690240e798340f5a15918d5c,offer viewed,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},0
12651,d1ede868e29245ea91818a903fec04c6,offer viewed,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},0
12652,102e9454054946fda62242d2e176fdce,offer viewed,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0
12653,02c083884c7d45b39cc68e1314fec56c,offer viewed,{'offer id': 'ae264e3637204a6fb9bb56bc8210ddfd'},0
12655,be8a5d1981a2458d90b255ddc7e0d174,offer viewed,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},0


In [5]:
#extract promotion id from value
offer_views = offer_views.reset_index()
offer_views = pd.concat([offer_views, pd.DataFrame((d for idx, d in offer_views['value'].iteritems()))], axis=1)
offer_views['id'] = offer_views['person']
offer_views.index = offer_views['id'] #set index equal to customer_id to match customers data
offer_views = offer_views.loc[:, ['offer id']]
display(HTML(offer_views.iloc[0:5].to_html()))
print(offer_views.shape)

,offer id
id,
389bc3fa690240e798340f5a15918d5c,f19421c1d4aa40978ebb69ca19b0e20d
d1ede868e29245ea91818a903fec04c6,5a8bc65990b245e5a138643cd4eb9837
102e9454054946fda62242d2e176fdce,4d5c57ea9a6940dd891ad53e9dbe8da0
02c083884c7d45b39cc68e1314fec56c,ae264e3637204a6fb9bb56bc8210ddfd
be8a5d1981a2458d90b255ddc7e0d174,5a8bc65990b245e5a138643cd4eb9837


(57725, 1)


Now we can reference the promotions to get the actual promotion.

In [6]:
display(HTML(portfolio.to_html()))

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [7]:
promotions = portfolio.drop(columns = ['channels','difficulty','duration','offer_type','reward'])
promotions = promotions.reset_index(inplace = False)
promotions = promotions.rename(columns={'index': 'offer code', 'id':'offer id'})
display(HTML(promotions.to_html()))


,offer code,offer id
0,0,ae264e3637204a6fb9bb56bc8210ddfd
1,1,4d5c57ea9a6940dd891ad53e9dbe8da0
2,2,3f207df678b143eea3cee63160fa8bed
3,3,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,4,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,5,2298d6c36e964ae4a3e7e9706d1fb8c2
6,6,fafdcd668e3743c1bb461111dcafc2a4
7,7,5a8bc65990b245e5a138643cd4eb9837
8,8,f19421c1d4aa40978ebb69ca19b0e20d
9,9,2906b810c7d4411798c6938adc9daaa5


In [8]:
offer_views['id'] = offer_views.index
offer_views = offer_views.merge(promotions, how='left', left_on='offer id', right_on='offer id')
offer_views.index = offer_views['id']
offer_views = offer_views.drop(columns = ['offer id', 'id'])
display(HTML(offer_views.iloc[0:10].to_html()))
print(offer_views.shape)

,offer code
id,
389bc3fa690240e798340f5a15918d5c,8
d1ede868e29245ea91818a903fec04c6,7
102e9454054946fda62242d2e176fdce,1
02c083884c7d45b39cc68e1314fec56c,0
be8a5d1981a2458d90b255ddc7e0d174,7
9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,9
8c7df0c393db488aac3e58b06a7ea5f9,5
e528ceb341964128aaf58a59733ec2af,5
262ad0fb526a4d53b572007da60cce24,8


(57725, 1)


For linear regression, we cannot have categorical data such as the offer code (even though they are numbers, the order is meaningless). To address this, we will make each promotion it's own column.

In [9]:
promotion_list = np.sort(offer_views['offer code'].unique())
print(promotion_list)


[0 1 2 3 4 5 6 7 8 9]


In [10]:
for prom in promotion_list:
    offer_views[prom] = prom
    offer_views[prom] = (offer_views[prom]==offer_views['offer code']).astype(int)
offer_views = offer_views.drop(columns = 'offer code')
display(HTML(offer_views.iloc[0:10].to_html()))       

,0,1,2,3,4,5,6,7,8,9
id,,,,,,,,,,
389bc3fa690240e798340f5a15918d5c,0,0,0,0,0,0,0,0,1,0
d1ede868e29245ea91818a903fec04c6,0,0,0,0,0,0,0,1,0,0
102e9454054946fda62242d2e176fdce,0,1,0,0,0,0,0,0,0,0
02c083884c7d45b39cc68e1314fec56c,1,0,0,0,0,0,0,0,0,0
be8a5d1981a2458d90b255ddc7e0d174,0,0,0,0,0,0,0,1,0,0
9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,0,0,0,0,0,0,0,0,1
8c7df0c393db488aac3e58b06a7ea5f9,0,0,0,0,0,1,0,0,0,0
e528ceb341964128aaf58a59733ec2af,0,0,0,0,0,1,0,0,0,0
262ad0fb526a4d53b572007da60cce24,0,0,0,0,0,0,0,0,1,0


In [11]:
#offer_views['id'] = offer_views.index
offer_views = offer_views.groupby(['id']).sum()

In [12]:
display(HTML(offer_views.iloc[0:1].to_html())) 

,0,1,2,3,4,5,6,7,8,9
id,,,,,,,,,,
0009655768c64bdeb2e877511632db8f,0,0,1,0,0,0,1,1,1,0


Here we can see that, for this specific customer, they viewed promotions 2, 6, 7 and 8 once each. 

In [13]:
print(offer_views.shape)

(16834, 10)


We can also see that *almost* every customer viewed at least 1 promotion (if you remember there are 17000 customers in the original dataset). 

Before continuing, let's cleanup our dataset so as to avoid confusion later. 

In [14]:
offer_views = offer_views.rename(columns={0: 'prom_0',1: 'prom_1',2: 'prom_2',3: 'prom_3',4: 'prom_4',
                                          5: 'prom_5',6: 'prom_6',7: 'prom_7',8: 'prom_8',9: 'prom_9'})
display(HTML(offer_views.iloc[0:1].to_html())) 

,prom_0,prom_1,prom_2,prom_3,prom_4,prom_5,prom_6,prom_7,prom_8,prom_9
id,,,,,,,,,,
0009655768c64bdeb2e877511632db8f,0,0,1,0,0,0,1,1,1,0


In [15]:
customers['cust_id'] = customers.index
offer_views['cust_id'] = offer_views.index
customers = customers.merge(offer_views, how='left')

In [16]:
customers.index = customers['cust_id']
customers = customers.drop(columns = 'cust_id')
display(HTML(customers.iloc[0:1].to_html())) 

,age,income,age of account,num purchases,amount per purchase,customer_segment,prom_0,prom_1,prom_2,prom_3,prom_4,prom_5,prom_6,prom_7,prom_8,prom_9
cust_id,,,,,,,,,,,,,,,,
e2127556f4f64592b11af22de27a7932,68,70,2,3,19.243333,3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0


We need to address the customers which did not view any promotions as they will have NAs as values. We can simply fill them in with zeros since a 0 reflects that a customer did not view the promotion, and the absence of the customer from the offer_views dataframe implies the customer did not view any promotions.

In [17]:
customers.isnull().values.any()

True

In [18]:
customers.fillna(0, inplace=True)
customers.isnull().values.any()

False

We now have a dataframe of numerical data that is ready for linear regression! 

Let's re-add an old feature, total_amount_purchased, as that will be the y value we will attempt to predict. Then we can save the data and move on to the next notebook.

In [19]:
customers['total_amount_purchased'] = customers['num purchases']*customers['amount per purchase']
customers = customers.drop(columns = ['num purchases', 'amount per purchase'])
display(HTML(customers.iloc[0:1].to_html())) 

,age,income,age of account,customer_segment,prom_0,prom_1,prom_2,prom_3,prom_4,prom_5,prom_6,prom_7,prom_8,prom_9,total_amount_purchased
cust_id,,,,,,,,,,,,,,,
e2127556f4f64592b11af22de27a7932,68,70,2,3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,57.73


In [20]:
import os

def make_csv(df, filename, data_dir):
    '''Merges features and labels and converts them into one csv file with labels in the first column.
       :param x: Data features
       :param y: Data labels
       :param file_name: Name of csv file, ex. 'train.csv'
       :param data_dir: The directory where files will be saved
       '''
    # make data dir, if it does not exist
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    df.to_csv(os.path.join(data_dir, filename), header=True, index=True)  
    
    # nothing is returned, but a print statement indicates that the function has run
    print('Path created: '+str(data_dir)+'/'+str(filename))

In [21]:
data_dir = 'regression_ready_data'
make_csv(customers,'customers.csv',data_dir)    

Path created: regression_ready_data/customers.csv


That concludes the further processing notebook. The next notebook is promotion evaluation through linear regression.

In [22]:
display(HTML(customers.iloc[0:1].to_html())) 

,age,income,age of account,customer_segment,prom_0,prom_1,prom_2,prom_3,prom_4,prom_5,prom_6,prom_7,prom_8,prom_9,total_amount_purchased
cust_id,,,,,,,,,,,,,,,
e2127556f4f64592b11af22de27a7932,68,70,2,3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,57.73
